# OscilloDSP の使い方

## 概要

OscilloDSP は、DSP などの組込プロセッサ上で動作するプログラムから、センサの値や演算値などをパソコンに送り、パソコン上の Python プログラムでオシロスコープのように視覚化するツールです。

![screen sample](img/screen_sample.png)

値データは、UART などの通信インターフェイスを使って、組込プロセッサからパソコンに送られます。

リアルタイムの測定データを全てパソコンに送出すると通信データの量が膨大となるため、OscilloDSP では、視覚化に必要な情報だけを送ることで、通信量を緩和しています。（実験では、2チャネルのデータを送るのに 100kbps 程度以下となっています。）

組込プロセッサとパソコンの間の通信データは、Google [Protocol Buffers](https://developers.google.com/protocol-buffers) という仕組を使って、系統的に符号化、また復号しています。

## PC シミュレーション

手元に DSP ボードがない場合でも、OscilloDSP の動作確認をできるように、[PC シミュレータ](../pcsim) が用意されています。
このシミュレータは POSIX（UNIX）用に書かれており、Linux や Mac OS などで動作します。
残念ながら、通信の仕組が異なるので、そのままでは Windows では動作しないかも知れません。

PC シミュレータで動作させる場合は、PC シミュレータの [main.c](../pcsim/main.c) と、[oscillo.c](../pcsim/oscillo.c) 中の関数```oscillo_get_demo1_value()``` と ```oscillo_get_demo2_value()``` を参考にしてください。
また、[oscillo.ipynb](oscillo.ipynb) の実行時に、セル「[OscilloDSP モジュールのロードと定数の定義](oscillo.ipynb#OscilloDSP-%E3%83%A2%E3%82%B8%E3%83%A5%E3%83%BC%E3%83%AB%E3%81%AE%E3%83%AD%E3%83%BC%E3%83%89%E3%81%A8%E5%AE%9A%E6%95%B0%E3%81%AE%E5%AE%9A%E7%BE%A9)」の中で```USE_PCSIM``` を ```True``` に設定してください。```oscillo.run_pcsim()``` という関数を実行すると、```pcsim``` が自動的にビルドされ、実行されます。

## DSP あるいは pcsim 側のプログラム

DSP 用のサンプルプロジェクトは、[ここ](../workspace/oscillodemo)にあります。
oscillodemo でも pcsim でも使い方は同等ですが、以下に DSP のサンプルプログラムを引用します。

### メインルーチン

TI DSP 用のメインルーチンでは、UART ペリフェラルの初期化をしています。
以下の例では、UART0 を 2083333 bps に設定しています。
この速度で、C6678 EVM で動作確認済みです。（対向のパソコンは、2000000 bps に設定します。数パーセントの誤差がありますが、作者の環境では動作しています。）

その後、DSP の RTOS（SYS/BIOS）のスケジューラを、```BIOS_start()``` で呼び出します。

```c
void main(void)
{
    init_cic_interrupt(UART0_INTNUM, UART0_CIC, CIC0_UART0INT_EV, CIC_OUT_CH);
    uart_init(UART0, uart_readbuf, uart_writebuf, UART_READBUF_LEN,
        UART_WRITEBUF_LEN, 2083333, 16, UART0_INTNUM);
    BIOS_start();
}
```

### OscilloDSP の初期化

タスク関数 ```func_task_main()``` 中の以下のコードが、OscilloDSP ライブラリの初期設定部分です。

```c
    oscillo_init(1e6, 1e-3);

    ch0 = oscillo_config_ch("input", "volts", -4.0, 5.0);
    ch1 = oscillo_config_ch("output", "amperes", -3.0, 3.0);
```

```oscillo_init()``` 関数の最初の引数は、サンプリングレート（Hz）で、次の引数は、パソコン側でオシロスコープ画面を表示する際の、
初期状態のスコープの時間幅（秒）です。
ここでは、サンプリングレートが 1MHz、スコープの幅を 1ミリ秒と指定しています。

サンプリングレートを指定する理由ですが、後のコードでサンプル値を OscilloDSP ライブラリに渡す際に、
関数の呼び出し毎に、どれだけの時間が経過しているか分からないと、オシロスコープ画面に時間軸を表示できないためです。

なお、上の例ではオシロスコープの 2チャネルぶんを初期化しています。
最初のチャネル ```ch0``` は、チャネルの名前を「input」とし（英数字を使って、任意の名前を指定できます）、単位を「volts」としています。
単位は、単純に「V」でも良いですが、画面に横向きに表示されるので、あえて volts としました。

その次の引数はそれぞれ、**このチャネル**のサンプル値の最小値と最大値です。
注意点としては、この範囲に収まらない値はクリッピングしてしまうので、ある程度余裕を見込んで設定する必要があります。
ただし範囲を広げすぎると、パソコンに送信するデータの分解能が落ちてしまう（量子化に使われるビット数が少なくなってしまう）ため、
あまり広くしないほうが良いでしょう。

最後の行はチャネル 1 の設定です。ここではチャネル名を「output」、単位はアンペアとしています。
もちろん自由な名前と単位を指定できます。

> **TIPS**
> 
> OscilloDSP の内部では、渡されたサンプル値は一時的に float 型で保管されますが、パソコンに送信する際に固定小数値に変換されます。
> この固定小数点の値は、Python 側のコードで ```oscillo.Oscillo()``` を呼び出す際に指定する ```quantize_bits```ビットに量子化されます。
> 
> ```quantize_bits``` は、8 あるいは 16などが妥当な値ですが、前者ではデータ量が削減され高速に動作するのに対して、16だとやや通信量が
> 増加します。
> パソコンの画面でグラフを表示する際に $2^{16}$ 段階の相違は視認できないので、通常は ```quantize_bits``` = 8 で十分でしょう。
> ただし、CSV ファイルなどにデータを保管するなど、8ビットより大きな量子化ビット数が欲しい場合は、16 に設定してみください。
>
> なお、（これはしないと思いますが、）もし 16より大きな量子化ビット数が必要な場合は、pcsim や DSP プログラムの [config.h](../pcsim/config.h)
> で、```typedef short buffer_t;``` という部分を、short から、より大きな整数型に変更する必要があります。

### サンプル値の渡し方

センサや計算値の値は、次のようなコードで渡すことができます。

```ch0``` は、先に ```oscillo_config_ch()``` から返された値で、これでチャネルを識別します。
2番目の引数は float 型で、センサ値や計算値を与えるものです。

```c
        oscillo_pass_one(ch0, oscillo_get_demo1_value(active));
        oscillo_pass_one(ch1, oscillo_get_demo2_value(true));
```

なお、```oscillo_get_demo1_value()``` および ```oscillo_get_demo2_value()``` というのは、
デモ用に用意されたサンプル値計算関数で、[oscillo.c](../pcsim/oscillo.c) で定義されています。
前者は

$$0.5 + 3\sin(2\pi ft) + 0.3\sin(2\pi ft) + {\rm ノイズ（}f \approx 2800 {\rm Hz）}$$

を、後者は振幅 1.5 の矩形波（ノイズあり）を返します。

いずれも、関数の中を見てみると参考になるでしょう。

> なお、```active```という変数ですが、定期的（4秒周期）に波形を出力したり停止したりするためのものです。
> これは、トリガ機能の試験のために用意したものです。

### パソコンとの通信

サンプル値をライブラリに渡すコードの後に、次のようなコードがあります。

```c
        oscillo_proc();
```

これは、パソコンから UART 経由のメッセージ（コマンド）があるかどうかを確認し、あればそのメッセージを処理するための関数です。
パソコン側の Python コードから波形データを渡すような指示があれば、データをパソコン側に送信します。

それほど頻繁に呼び出す必要はありませんが、例えば 1秒間に 3回以上、再描画したい場合は、
この関数をその頻度（1秒間に 3回）以上で呼び出す必要があります。
この関数は「ブロック」しません。必要な処理がなければ（パソコンからメッセージが来てなければ）すぐにリターンします。

なお、OscilloDSP ライブラリは [thread-safe](https://en.wikipedia.org/wiki/Thread_safety) に設計されていませんので、
```oscillo_pass_one()```と ```oscillo_proc()``` を、RTOS（SYS/BIOS）の異なるタスクから呼び出すことはできません
（動作が保証されません）。
thread-safe にしたい場合は、ラッパー関数などを実装してください。（その必要があるケースは少ないと思います。）


## パソコン側のプログラム

パソコン上のオシロスコープ視覚化には、Python 言語のプログラムを使います。
また、Python をコマンドラインで実行するのではなく、[Jupyter Notebook](https://jupyter.org/) と、さらに以下のツールを使用します。

- [Matplotlib](https://matplotlib.org/)
- [ipywidgets](https://ipywidgets.readthedocs.io/)
- [ipympl](https://github.com/matplotlib/ipympl)

順に簡単に説明します。

### Jupyter Notebook

ウェブブラウザ上で、（この説明書のように）ドキュメントとプログラムコードを一元的に集め、手元のパソコンやネット上のサーバーでプログラムを動作させることのできる、「動的な」レポートを作成するためのツールです。

### Matplotlib

Python で数学的、あるいは技術的なグラフをプロットするためのライブラリです。
Jupyter Notebook なしでも利用できます。

### ipywidgets

Jupyter Notebook 上で、グラフィカルユーザーインターフェイス（GUI）の部品と、Python のプログラムを連携させるためのライブラリです。

### ipympl

ipywidgets と Matplotlib を統合するためのライブラリです。

## 実際の使い方

### 初期設定

既に説明したように、PC シミュレータ pcsim を使う場合には、```USE_PCSIM``` を ```True``` にしてください。
その場合は、それ以外の設定は不要です。

> pcsim を使う場合は、```UART_BITRATE``` は 115200 bps 以下の、一般的なシリアル通信速度に
> してください。

実際に DSP など組込プロセッサと通信させる場合には、予め DSP ボードなどとパソコンを UART で接続しておきます。
お勧めの接続方法は、FTDI 社の FT232R や FT2232H といった USB-UART ブリッジインターフェイスを使うものです。
これならば、組込プロセッサの能力に依りますが、2Mbps 程度の高速な通信が可能です。

通常のシリアルインターフェイスを使う場合には、OS に依って異なりますが、```UART_DEVICE```に ```COM1``` や
```/dev/ttyUSB0``` などのように指定します。

FTDI 社の USB-UART ブリッジを使う場合は、予め [PyFtdi](https://github.com/eblot/pyftdi) の
[インストール方法サイト](https://eblot.github.io/pyftdi/installation.html) に従ってインストールしておきます。

> [installation.ipynb](installation.ipynb) も参考にしてください。

そして、```UART_DEVICE``` には（PyFtdi 特有の） ```ftdi://ftdi:232:FTGYDM6A/1``` のようなデバイス名を指定をします。
どのようなデバイス名を指定したら良いか分からない場合は、仮に ```ftdi:```（コロンを忘れずに）と指定しておきます。
そして「オシロのセットアップ」の下のコードまで実行すると、オシロスコープ画面の下に、実際にあなたの環境で利用できるデバイス名が表示されます。
TI C6657 EVM や C6678 EVM ボード上の FT2232H を使う場合には、2つ表示されたうちの後者のほうを指定してください。
（前者は、ボード上の JTAG エミュレータとして接続されています。）

Python のコードを実行する前に、組込プロセッサ側のプログラムをスタートしておきます。

続いて、[oscillo.ipynb](oscillo.ipynb) の、上から順にセルを実行していきます。
「オシロのスタート」の下のコードを実行すると、オシロスコープが動き始めます。

### オシロスコープの操作

オシロスコープ画面の左側には波形が表示されます。
複数チャネルある場合、チャネル毎に色が異なり、

- チャネル 0: 黄色
- チャネル 1: 水色
- チャネル 2: ピンク
- チャネル 3: 濃い青
- それ以外: 黒（プログラムコードを修正することで変更可能）

のようになっています。（これは、手持ちの Rigol 社のオシロスコープに合わせました。）

画面右のプルダウンメニューやスライダ（つまみ）、ボタンの操作方法を説明します。

#### Active Channel

既に説明したように、DSP 側の ```oscillo_config_ch()``` では、チャネル毎に物理単位や値の範囲を個別に設定できます。
そのため、オシロスコープの縦軸の値や単位は、チャネル毎に異なることがあります。
Active Channel で任意のチャネルを選択することで、この縦軸表示を切り替えることができます。
例えば、チャネル 0 の単位がボルト（volts）の場合は、Active Channel で 0 を選ぶことで、
正しい縦軸が表示されます。

#### Trigger Mode

3つのモードから選択できます。
これは、通常のオシロスコープと同等と思います。

- **Auto**: 自動モード（トリガが掛かっているときは、トリガに従って表示します。トリガが掛かっていないときは、常に最新のデータを表示し続けます。）
- **Normal**: ノーマルモード（トリガが掛かっていない場合は、画面の更新が止まります。DSP からデータが送られません。）
- **Single**: シングル（ワンショット）モード（トリガが掛かると画面の更新が止まります。「Reset」ボタンを押すと、再度トリガ待ちになります。）

> **学生さん向けの課題**
> 
> 現在、トリガに使用できるチャネルは、チャネル 0 固定になっています。
> Python のコード [oscillo.py](oscillodsp/oscillo.py) を変更することで機能追加できます。
>
> また、現在はトリガタイプはアップエッジ（rising edge）固定になっています。
> Python のコードと、DSP のコードを少し変更することで、ダウンエッジに対応可能です。

#### Trigger Status

現在のトリガ状態です。ステータスが点滅中は、トリガが掛かっていないことを意味します。

#### Trigger Level

トリガレベルです。単位は物理量（ボルトやアンペア）です。窓右の「矢印ボタン」でステップ調整することができますが、
窓に直接値を入力することもできます。（後者のほうが一般性があると思います。）

#### Horizontal Scale

時間軸のスケール調整です。
右に動かすと拡大できますが、サンプリングレートよりも高く指定しても無意味です。
（例: サンプリングレートが 1MHz のとき、スケールを 1マイクロ秒に設定すると、一画面に 1サンプル分しか表示されません。）

なお左に動かすと縮小できますが、DSP 中の（環状）バッファ ```buffer[]``` のサイズ ```LEN_BUFFER``` よりも広げることはできません。
たとえば、サンプリングレートが 1MHz で ```LEN_BUFFER``` が 16384 のとき、バッファ中に 16.384ミリ秒分のサンプルしか保持できませんので、
スケールをそれ以上に広げる（スライダを左に動かす）ことはできません。

#### Scale, Pos

オシロスコープ右の 2つのスライダが Scale と Pos です。
これを使って、現在の Active Channel の縦軸スケールと、オフセットを調整できます。
Pos の丸いノブの位置が、物理量のゼロ軸に相当しますので、たとえばチャネルの物理単位がボルトで、ノブを画面下 1/3 のところに持ってくると、
画面下 1/3 のところがゼロボルトになります。

普段から実際のオシロスコープを利用している方は、複数のチャネルの波形が重ならないように（このページの最初の画面例参照）調整することが
できますし、そうでない方は、例えば同じ物理量（2つのチャネルが、いずれもボルト）なのであれば、画面の中央にゼロボルトを表示したほうが、
分かりやすいかも知れません。（ただし、```oscillo_config_ch()``` の最小値、最大値設定がチャネル毎に異なる場合は、値を読み誤らないように、
Active Channel でチャネルを選択してから縦軸を見るようにしてください。）

#### Run/Stop

オシロスコープを停止したり再開したりできます。
ただし、停止中はプログラムが止まっていますので、GUI による設定は働きません。

#### Save as Image File

現在の表示波形を PNG 形式で保存します。
Jupyter Notebook を起動したディレクトリに保存されます。

> **注意点**
>
> 今回は、Matplotlib のバージョン 3.1.3 を利用していますが、より新しいバージョンの Matplotlib では仕様が異なり、
> オシロプログラム実行中の画面保存ができません。
> どうしても 3.1.3 より新しいバージョンを用いたい場合は、一度オシロスコープを Run/Stop ボタンで停めてから
> Save as Image File してみてください。（ただし、詳細は未確認です。）

#### Save as CSV File

同様に、CSV ファイルを保存します。

## さまざまな定数値の変更方法

プログラム中の定数を変更することで、動作時の制約を変更することができます。

### 最大チャネル数

デフォルトでは 2となっています。ファイル [protobuf/oscillodsp.options](protobuf/oscillodsp.options) 中の

```
ConfigReply.chconfig            max_count: 2
WaveGroup.wave                  max_count: 2
```

の値を**両方を**変えることで、変更できます。
なお、この Protocol Buffers の定義ファイルを変更した場合は、ディレクトリ ```protobuf``` で ```make``` コマンドを実行することで、
以下のファイルを更新（再構成）する必要があります。

- [hostapp/oscillodsp/oscillodsp_pb2.py](oscillodsp/oscillodsp_pb2.py)
- [pcsim/oscillodsp.pb.c](../pcsim/oscillodsp.pb.c)
- [pcsim/oscillodsp.pb.h](../pcsim/oscillodsp.pb.h)
- [workspace/oscillodemo/oscillodsp.pb.c](../workspace/oscillodemo/oscillodsp.pb.c)
- [workspace/oscillodemo/oscillodsp.pb.h](../workspace/oscillodemo/oscillodsp.pb.h)

### 波形サンプル数

DSP からパソコンに一度に渡す波形サンプル数は、デフォルトでは 500に設定されています。
これでも視覚化上問題ないと思いますが、1000 に増やしたい場合は、
ファイル [protobuf/oscillodsp.options](protobuf/oscillodsp.options) 中の

```
Wave.samples                    max_count: 500
```

の値を 1000に変えることで、変更できます。（データ通信量が 2倍になります。）
この場合も、上記と同様に生成ファイルを更新（再構成）してください。

### DSP 中のバッファサイズ

DSP プログラム中の関数 ```oscillo_pass_one()```で渡す値は、パソコンに転送する前に環状バッファ ```buffer[][]```
に保管されます。
このバッファサイズはデフォルトで 16384サンプルとなっていますが、変更したい場合は、
pcsim あるいは DSP デモコード中の ```config.h``` にて

```c
enum oscillo {
    LEN_BUFFER = 16384,
};
```

の値を変更してください。
この値を大きくすると、オシロスコープの Horizontal Scale つまみを、より左側に持っていけるようになります。
（つまり、サンプル値を過去に遡ってより広く見られるようになります。）

### パソコンに送るサンプル値の量子化ビット数

上の説明「[OscilloDSP の初期化](#OscilloDSP-%E3%81%AE%E5%88%9D%E6%9C%9F%E5%8C%96)」にある TIPS を参照してください。